# Customer data wrangling, consistency checks & merge 
- customer data set wrangling
- customer data set consistency checks
- merging of customer data set with latest ords_prods version

In [23]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [24]:
# Import customer data set
customers = pd.read_csv(r'/Users/OldBobJulia/Desktop/CF/Course/4. Python/Instacart Basket Analysis/02 Data/Original data/customers.csv')

In [1]:
# I removed the following outputs in this copy of the notebook because they contained PII. 

In [26]:
customers.shape

(206209, 10)

In [28]:
# Wrangle: rename columns first name, surname, gender, state, age for consistency
customers.rename(columns = {'First Name': 'first_name', 'Surnam': 'surname', 'Gender': 'gender', 'STATE': 'state', 'Age': 'age'}, inplace = True)

In [30]:
# Wrangle: drop column date_joined. We have info on prior_orders in prods_orders_merge data set. 
customers = customers.drop(columns = ['date_joined'])


In [32]:
# Check data types
customers.dtypes

user_id          int64
first_name      object
surname         object
gender          object
state           object
age              int64
n_dependants     int64
fam_status      object
income           int64
dtype: object

# Data types seem fine. Wrangling complete. 

In [33]:
# Data consistency checks: find mixed data type columns
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

first_name


In [34]:
customers['first_name'].value_counts(dropna=False)

NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Eugene       197
Merry        197
Garry        191
David        186
Ned          186
Name: first_name, Length: 208, dtype: int64

In [35]:
customers.dtypes.value_counts(dropna=False)

object    5
int64     4
dtype: int64

# First_name has NaN values but this does not influence the analysis because we won't analyse first names. However, the observations themselves remain meaningful. So I just keep them as they are.

In [36]:
# Change first_name column into object
customers['first_name'] = customers['first_name'].astype('str')

In [37]:
# Check for mixed data columns again
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

In [38]:
# Check data types
customers.dtypes

user_id          int64
first_name      object
surname         object
gender          object
state           object
age              int64
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [39]:
# Data consistency: check for missing values
customers.isnull().sum()

user_id         0
first_name      0
surname         0
gender          0
state           0
age             0
n_dependants    0
fam_status      0
income          0
dtype: int64

# It seems we have no missing data (the kind recognizable by pandas) nor duplicates

In [40]:
# Check summary statistics
customers.describe()

,user_id,age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [41]:
# I find the max. income slightly suspicious. Research shows it's a very high but not unrealistic salary. 
customers['income'].value_counts(dropna=False)

95891     10
57192     10
95710     10
94809      9
97532      9
          ..
139481     1
152861     1
464181     1
228664     1
28658      1
Name: income, Length: 108012, dtype: int64

In [42]:
# Create subset of potentially unrealistic incomes
df_unreal = customers[customers['income'] >= 300000]

In [43]:
df_unreal.head()

,user_id,first_name,surname,gender,state,age,n_dependants,fam_status,income
434,159362,Tina,Shannon,Female,Missouri,74,3,married,372334
979,200930,Charles,Nichols,Male,South Carolina,60,1,married,300913
991,136298,Kevin,Ortega,Male,New Mexico,47,3,married,433206
1903,89592,Katherine,Abbott,Female,Minnesota,22,0,single,317122
2018,33595,Jose,Castaneda,Male,Mississippi,65,2,married,577728


In [44]:
df_unreal.shape

(382, 9)

# These salaries seem pretty legit. 

In [45]:
# Create new subset of potentially unrealistic incomes.
df_unreal_2 = customers[customers['income'] >= 500000]

In [46]:
df_unreal_2.shape

(70, 9)

In [1]:
# Again I remove some outcomes from here because of PII

In [49]:
df_unreal_2['income'].value_counts(dropna=False)

511616    1
569810    1
553569    1
514782    1
536156    1
         ..
543925    1
531126    1
528184    1
574425    1
506877    1
Name: income, Length: 70, dtype: int64

# No repeated numbers. Salaries seem legit. 

In [50]:
# Further consistency checks
customers['gender'].value_counts(dropna=False)

Male      104067
Female    102142
Name: gender, dtype: int64

In [51]:
customers['state'].value_counts(dropna=False)

Georgia                 4044
Illinois                4044
Connecticut             4044
District of Columbia    4044
Arizona                 4044
Indiana                 4044
Colorado                4044
Delaware                4044
Florida                 4044
Iowa                    4044
Hawaii                  4044
Arkansas                4044
Alaska                  4044
California              4044
Idaho                   4044
Alabama                 4044
Oklahoma                4043
South Carolina          4043
Wisconsin               4043
Oregon                  4043
South Dakota            4043
Michigan                4043
Rhode Island            4043
Vermont                 4043
Kansas                  4043
Utah                    4043
Maryland                4043
New Hampshire           4043
Missouri                4043
Nevada                  4043
Virginia                4043
Kentucky                4043
Nebraska                4043
Montana                 4043
New York      

# We have a similar amount of female/male.

# The number of customers per state are near identical. This must have been done on purpose. Comparing customer counts between states is therefore useless.   

In [52]:
customers['age'].value_counts(dropna=False)

19    3329
55    3317
51    3317
56    3306
32    3305
      ... 
65    3145
25    3127
66    3114
50    3102
36    3101
Name: age, Length: 64, dtype: int64

In [53]:
customers['n_dependants'].value_counts(dropna=False)

0    51602
3    51594
1    51531
2    51482
Name: n_dependants, dtype: int64

# In fact, demographics such as age and n_dependants also seem chosen to be represented in equal numbers. 

In [54]:
customers['fam_status'].value_counts(dropna=False)

married                             144906
single                               33962
divorced/widowed                     17640
living with parents and siblings      9701
Name: fam_status, dtype: int64

In [55]:
customers['surname'].value_counts(dropna=False)

Hamilton      252
Randall       248
Pennington    243
Lamb          243
Barnett       242
             ... 
Poole         172
Bauer         166
Pearson       164
Payne         163
Jordan        162
Name: surname, Length: 1000, dtype: int64

In [56]:
customers.shape

(206209, 9)

In [57]:
# Import prods_ords_merge
prods_ords_merge = pd.read_pickle(r'/Users/OldBobJulia/Desktop/CF/Course/4. Python/Instacart Basket Analysis/02 Data/Prepared data/orders_products_merged_070821.pkl')

In [58]:
# Check 
prods_ords_merge.shape

(32404859, 22)

In [59]:
customers.shape

(206209, 9)

In [60]:
# Compare column names 
prods_ords_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,_merge,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,spending_mean,spending_flag,order_frequency,order_frequency_flag
0,2539329,1,1,2,8,0.0,196,1,0,Soda,...,both,Mid-range product,Regularly busy,Average orders,15,Regular customer,7.923932,Low spender,14.0,Regular customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Average orders,15,Regular customer,7.923932,Low spender,14.0,Regular customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Average orders,15,Regular customer,7.923932,Low spender,14.0,Regular customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Average orders,15,Regular customer,7.923932,Low spender,14.0,Regular customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Most orders,15,Regular customer,7.923932,Low spender,14.0,Regular customer


In [62]:
# Checking data type key user_id
prods_ords_merge.dtypes

order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
product_name                object
department_id                int64
prices                     float64
_merge                    category
price_range_loc             object
busiest_day                 object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
spending_mean              float64
spending_flag               object
order_frequency            float64
order_frequency_flag        object
dtype: object

# User_id dtype is int64 in both cases

In [63]:
# Drop previous _merge indicator column
prods_ords_merge = prods_ords_merge.drop(columns= ['_merge'])

In [64]:
pd.options.display.max_columns = None

In [65]:
prods_ords_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,department_id,prices,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,spending_mean,spending_flag,order_frequency,order_frequency_flag
0,2539329,1,1,2,8,0.0,196,1,0,Soda,7,9.0,Mid-range product,Regularly busy,Average orders,15,Regular customer,7.923932,Low spender,14.0,Regular customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,7,9.0,Mid-range product,Least busy,Average orders,15,Regular customer,7.923932,Low spender,14.0,Regular customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,7,9.0,Mid-range product,Least busy,Average orders,15,Regular customer,7.923932,Low spender,14.0,Regular customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,7,9.0,Mid-range product,Least busy,Average orders,15,Regular customer,7.923932,Low spender,14.0,Regular customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,7,9.0,Mid-range product,Least busy,Most orders,15,Regular customer,7.923932,Low spender,14.0,Regular customer


In [66]:
# Perform left inner join on table A (prods_ords_merge) so that I have all orders, even those of which we don't have customer data (if any)
df_merged = prods_ords_merge.merge(customers, on = ['user_id'], indicator = True)

In [68]:
df_merged.shape

(17027529, 30)

# I used the default inner join merge. So all new rows in data set are those where both sides match. 

In [70]:
df_merged['_merge'].value_counts(dropna=False)

both          17027529
left_only            0
right_only           0
Name: _merge, dtype: int64

In [71]:
# Create path and export
path = r'/Users/OldBobJulia/Desktop/CF/Course/4. Python/Instacart Basket Analysis/02 Data'

In [72]:
df_merged.to_pickle(os.path.join(path, 'Prepared data', 'df_merged070821.2.pkl'))